## Business Problem: 
###### One of the most significant issues in e-commerce is the accurate calculation of the post-purchase ratings given to products. Solving this problem means providing more customer satisfaction for the e-commerce site, highlighting the product for the sellers, and ensuring a seamless shopping experience for the buyers. Another problem is the correct sorting of product reviews. Misleading reviews can directly affect the product's sales, causing both financial loss and customer loss. Solving these two primary problems will increase sales for e-commerce sites and sellers while ensuring a smooth buying journey for customers.

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 10)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
df_ = pd.read_csv('/Users/yagizkarakaya/Desktop/DSMLBootcamp/Measurement_Problems/Rating Product&SortingReviewsinAmazon/amazon_review.csv')


In [4]:
df = df_.copy()

In [5]:
df.shape

(4915, 12)

In [6]:
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


###### how many products are there


In [8]:
df['asin'].value_counts()


B007WTAJTO    4915
Name: asin, dtype: int64

###### there is only 1 product. let's just look at its average


In [9]:
df['overall'].mean()


4.587589013224822

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB


In [11]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'], dayfirst=True)
current_date = pd.to_datetime('2014-12-09 00:00:00')
df['days'] = (current_date - df['reviewTime']).dt.days
df['days'].describe().T

count   4915.00000
mean     438.36704
std      209.43987
min        2.00000
25%      282.00000
50%      432.00000
75%      602.00000
max     1065.00000
Name: days, dtype: float64

In [12]:
print(df.loc[df["day_diff"] <= df["day_diff"].quantile(0.25), "overall"].mean())
print(df.loc[(df["day_diff"] > df["day_diff"].quantile(0.25)) & (df["day_diff"] <= df["day_diff"].quantile(0.50)), "overall"].mean())
print(df.loc[(df["day_diff"] > df["day_diff"].quantile(0.50)) & (df["day_diff"] <= df["day_diff"].quantile(0.75)), "overall"].mean())
print(df.loc[(df["day_diff"] > df["day_diff"].quantile(0.75)), "overall"].mean())

4.6957928802588995
4.636140637775961
4.571661237785016
4.4462540716612375


### Determination of time-based average weights


In [13]:
def time_based_weighted_average(dataframe, w1=50, w2=25, w3=15, w4=10):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.25), "overall"].mean() * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.25)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.50)), "overall"].mean() * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.50)) & (dataframe["day_diff"] <= dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile(0.75)), "overall"].mean() * w4 / 100

In [14]:
time_based_weighted_average((df))

4.637306192407316

In [15]:
df["overall"].mean()


4.587589013224822

### Task 2: Specify the 20 Reviews to be Displayed on the Product Detail Page for the Product.

In [16]:
df.head()


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,days
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0,139
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,410
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0,716
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,383
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0,514


In [17]:
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]


In [18]:
df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]

In [19]:
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.00000,Four Stars,0,0,0,2014-07-23
1,0mie,5.00000,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.00000,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.00000,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.00000,best deal around,0,0,0,2013-07-13


##### To calculate the scores of score_pos_neg_diff, 
##### score_average_rating, and wilson_lower_bound, 
##### and add them to the data

In [20]:
def score_up_down_diff(up, down):
    return up - down

In [21]:
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [22]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
    The lower bound of the confidence interval calculated for the Bernoulli parameter p is accepted as the WLB score.
    The calculated score is used for product ranking.
    Note:
        If the scores are between 1-5, they can be converted to the appropriate Bernoulli values by marking 1-3 as negative and 4-5 as positive.
        This approach brings some problems along with it. Therefore, Bayesian average rating should be calculated.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [23]:
df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_pos_neg_diff", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37
315,"Amazon Customer ""johncrea""",5.00000,Samsung Galaxy Tab2 works with this card if re...,38,10,48,2012-08-13,28


In [24]:
# score_average_rating
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_average_rating", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000
435,Anthony L cate,5.00000,Love the extra storage,1,0,1,2012-07-24,1,1.00000
2901,luis,5.00000,Awesome and fast card :),1,0,1,2013-05-13,1,1.00000
2204,"jbwam ""jbwam""",2.00000,Sandisk will replace failures due to bad batch...,1,0,1,2013-06-14,1,1.00000
2206,JCBiker,5.00000,Great card,1,0,1,2013-10-31,1,1.00000
3408,"Neng Vang ""Neng2012""",5.00000,working no problem,1,0,1,2013-07-25,1,1.00000


In [25]:
# wilson_lower_bound
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)
df.head()


,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,NaN,4.00000,Four Stars,0,0,0,2014-07-23,0,0.00000,0.00000
1,0mie,5.00000,MOAR SPACE!!!,0,0,0,2013-10-25,0,0.00000,0.00000
2,1K3,4.00000,nothing to really say....,0,0,0,2012-12-23,0,0.00000,0.00000
3,1m2,5.00000,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21,0,0.00000,0.00000
4,2&amp;1/2Men,5.00000,best deal around,0,0,0,2013-07-13,0,0.00000,0.00000


##### Specify 20 Comments

In [27]:
df.sort_values("score_pos_neg_diff", ascending=False).head(20)


,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.80576,0.73214
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.75229,0.66359
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52,0.88235,0.78465
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37,0.78462,0.67033
315,"Amazon Customer ""johncrea""",5.00000,Samsung Galaxy Tab2 works with this card if re...,38,10,48,2012-08-13,28,0.79167,0.65741


In [28]:
df.sort_values("score_average_rating", ascending=False).head(20)


,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000,0.20655
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000,0.20655
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000,0.20655
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000,0.20655
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000,0.20655
435,Anthony L cate,5.00000,Love the extra storage,1,0,1,2012-07-24,1,1.00000,0.20655
2901,luis,5.00000,Awesome and fast card :),1,0,1,2013-05-13,1,1.00000,0.20655
2204,"jbwam ""jbwam""",2.00000,Sandisk will replace failures due to bad batch...,1,0,1,2013-06-14,1,1.00000,0.20655
2206,JCBiker,5.00000,Great card,1,0,1,2013-10-31,1,1.00000,0.20655
3408,"Neng Vang ""Neng2012""",5.00000,working no problem,1,0,1,2013-07-25,1,1.00000,0.20655


In [29]:
df.sort_values("wilson_lower_bound", ascending=False).head(20)


,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52,0.88235,0.78465
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.80576,0.73214
3807,R. Heisler,3.00000,"Good buy for the money but wait, I had an issue!",22,3,25,2013-02-27,19,0.88000,0.70044
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37,0.78462,0.67033
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.75229,0.66359
